In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('cos_e', 'v1.11')

# Load the tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=5)  # Assuming 5 possible answer choices

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [2]:
# def preprocess_function(examples):
#     inputs = ["question: " + q + " answer: " + " ".join(choices) for q, choices in zip(examples['question'], examples['choices'])]
#     targets = ["answer: " + answer + " explanation: " + explanation for answer, explanation in zip(examples['answer'], examples['abstractive_explanation'])]
#     model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding='max_length')
#     labels = tokenizer(targets, max_length=256, truncation=True, padding='max_length')
#     model_inputs['labels'] = labels['input_ids']
#     return model_inputs

In [3]:
def preprocess_function(examples):
    inputs = ["question: " + q + " [SEP] " + " ".join(choices) for q, choices in zip(examples['question'], examples['choices'])]
    # Map the answer keys from string to the index of the correct choice
    labels = [choices.index(answer) for choices, answer in zip(examples['choices'], examples['answer'])]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding='max_length')
    model_inputs['labels'] = labels
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [4]:
tokenized_datasets['validation']['labels']

[2,
 0,
 2,
 3,
 0,
 0,
 4,
 1,
 1,
 4,
 0,
 3,
 0,
 0,
 2,
 4,
 3,
 1,
 0,
 2,
 1,
 4,
 0,
 1,
 4,
 1,
 3,
 4,
 3,
 4,
 3,
 0,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 0,
 3,
 1,
 4,
 1,
 2,
 4,
 3,
 4,
 2,
 0,
 3,
 4,
 3,
 4,
 3,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 2,
 0,
 3,
 3,
 1,
 3,
 4,
 4,
 3,
 3,
 4,
 1,
 2,
 3,
 4,
 4,
 3,
 2,
 2,
 4,
 3,
 3,
 4,
 4,
 2,
 1,
 4,
 2,
 2,
 1,
 1,
 1,
 4,
 1,
 2,
 3,
 4,
 4,
 4,
 0,
 0,
 1,
 0,
 4,
 0,
 2,
 2,
 4,
 2,
 2,
 0,
 0,
 4,
 0,
 3,
 1,
 1,
 2,
 1,
 0,
 2,
 2,
 1,
 4,
 2,
 0,
 0,
 3,
 2,
 0,
 3,
 1,
 1,
 1,
 1,
 2,
 2,
 3,
 1,
 0,
 3,
 2,
 3,
 4,
 1,
 2,
 4,
 1,
 2,
 4,
 0,
 2,
 1,
 0,
 1,
 1,
 4,
 3,
 0,
 1,
 3,
 3,
 4,
 1,
 0,
 1,
 1,
 1,
 4,
 1,
 0,
 2,
 0,
 2,
 1,
 3,
 0,
 0,
 4,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 1,
 3,
 4,
 1,
 3,
 3,
 1,
 0,
 1,
 0,
 4,
 1,
 1,
 3,
 4,
 0,
 1,
 1,
 2,
 4,
 2,
 4,
 1,
 0,
 3,
 2,
 3,
 1,
 2,
 4,
 1,
 3,
 1,
 1,
 4,
 1,
 2,
 0,
 4,
 2,
 1,
 0,
 1,
 2,
 4,
 3,
 1,
 4,
 1,
 1,


In [5]:
from torch.utils.data import DataLoader, TensorDataset

def convert_to_tensors(dataset):
    input_ids = torch.tensor(dataset['input_ids'])
    attention_mask = torch.tensor(dataset['attention_mask'])
    labels = torch.tensor(dataset['labels'])
    return TensorDataset(input_ids, attention_mask, labels)

train_dataset = convert_to_tensors(tokenized_datasets["train"])
eval_dataset = convert_to_tensors(tokenized_datasets["validation"])

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=128)
eval_loader = DataLoader(eval_dataset, batch_size=128)


In [6]:
# # Define optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)

# # Training loop
# def train_loop(model, loader, optimizer):
#     model.train()
#     total_loss = 0

#     for batch in loader:
#         optimizer.zero_grad()
#         inputs = {
#             'input_ids': batch[0].to(device),
#             'attention_mask': batch[1].to(device),
#             'labels': batch[2].to(device)
#         }
#         outputs = model(**inputs)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     return total_loss / len(loader)

# # Validation loop
# def validate_loop(model, loader):
#     model.eval()
#     total_loss = 0

#     with torch.no_grad():
#         for batch in loader:
#             inputs = {
#                 'input_ids': batch[0].to(device),
#                 'attention_mask': batch[1].to(device),
#                 'labels': batch[2].to(device)
#             }
#             outputs = model(**inputs)
#             loss = outputs.loss
#             total_loss += loss.item()

#     return total_loss / len(loader)

# # Training and validation
# num_epochs = 3
# for epoch in range(num_epochs):
#     train_loss = train_loop(model, train_loader, optimizer)
#     val_loss = validate_loop(model, eval_loader)
#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     print(f"Train Loss: {train_loss:.4f}")
#     print(f"Validation Loss: {val_loss:.4f}")



In [7]:
# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
def train_loop(model, loader, optimizer):
    model.train()
    total_loss = 0

    for batch in tqdm(loader, desc="Training"):
        optimizer.zero_grad()
        inputs = {
            'input_ids': batch[0].to(device),
            'attention_mask': batch[1].to(device),
            'labels': batch[2].to(device)
        }
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)


/home/manavi/myenv/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
from sklearn.metrics import accuracy_score

# Validation loop
def validate_loop(model, loader):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            inputs = {
                'input_ids': batch[0].to(device),
                'attention_mask': batch[1].to(device),
                'labels': batch[2].to(device)
            }
            outputs = model(**inputs)
            loss = outputs.loss
            total_loss += loss.item()

            preds = outputs.logits.argmax(dim=-1).cpu().numpy()
            labels = batch[2].cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)

    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)

    return avg_loss, accuracy




In [9]:

from tqdm import tqdm# Training and validation
num_epochs = 3
for epoch in range(num_epochs):
    train_loss = train_loop(model, train_loader, optimizer)
    val_loss, val_accuracy = validate_loop(model, eval_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")

Validation: 100%|██████████| 10/10 [00:05<00:00,  2.00it/s]


Epoch 1/3
Train Loss: 1.6149
Validation Loss: 1.6101
Validation Accuracy: 0.2088


Validation: 100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


Epoch 2/3
Train Loss: 1.6118
Validation Loss: 1.6150
Validation Accuracy: 0.2113


Validation: 100%|██████████| 10/10 [00:05<00:00,  1.99it/s]

Epoch 3/3
Train Loss: 1.6104
Validation Loss: 1.6110
Validation Accuracy: 0.2039
